**HWR Berlin**     
**Data Science Group Project**     
**Elo Merchant Category Recommendation**       
**30.01.2019**

Fabian Petsche, Matr. Nr: 463511

Oksana Hrytsiv, Matr. Nr: 741766

Fabian Asal, Matr. Nr: 734883  

## Notebook Content:

  1. [Introduction](#1)
 
  1.1 [Elo](#1.1)
  
  1.2 [Problem Statement](#1.2)
  
  
 2. [EDA](#2)
 
    2.1 [Data Load](#2.1)
   
    2.2 [Visualization](#2.2)
    
    2.3 [Feature Engineering](#2.3)
    
    2.4 [Pre-Processing](#2.4)
    
       2.4.1     [Missing Values](#2.4.1)
       
       2.4.2     [Scaling](#2.4.2)
       
       2.4.3     [Outliers](#2.4.3)
    

 3. [Model Training](#3)
 
     3.1 [Linear Models](#3.1)
     
     3.2 [Sklearn Tree's](#3.2)
       
       3.2.1     [GridSearch](#3.2.1)
      
     3.3 [LightGBM Tree's](#3.3)
       
       3.3.1     [Bayesian Optimization](#3.3.1)
       
       3.3.2     [Feature Importance](#3.3.2)
     
     3.4 [Models without Outliers](#3.4)
   
 4. [Conclusion](#4)

<a id="1"></a> <br>
# 1. Introduction
<a id="1.1"></a> <br>
## 1.1 Elo

### - Competition description: 

> Imagine being hungry in an unfamiliar part of town and getting restaurant recommendations served up, based on your personal preferences, at just the right moment. The **recommendation** comes with an attached discount from your credit card provider for a local place around the corner!
> 
> Right now, Elo, one of the largest payment brands in Brazil, has built partnerships with merchants in order to **offer promotions or discounts to cardholders**. But do these promotions work for either the consumer or the merchant? Do customers enjoy their experience? Do merchants see repeat business? Personalization is key.
> 
> Elo has built machine learning models to understand the most important aspects and preferences in their customers’ lifecycle, from food to shopping. But so far none of them is **specifically tailored** for an individual or profile. This is where you come in.
> 
> In this competition, Kagglers will **develop algorithms to identify and serve the most relevant opportunities to individuals, by uncovering signal in customer loyalty**. Your input will improve customers’ lives and help Elo reduce unwanted campaigns, to create the right experience for customers.

<a id="1.2"></a> <br>
## 1.2 Problem Statement



### - What is Loyalty?:

 According to the 'Data_Dictionary':  

- "*Loyalty is a **numerical score calculated 2 months after historical and evaluation period***".

'Valeria Elo' (Topic Author)_

- *"This competition is not a traditional collaborative filtering/recommendation system challenge since we don’t have a direct definition of what is great merchant or experience. Despite that, we want to **link historical information of merchants visits and purchases to a loyalty score**, which is basically a **business metric** that **considered both future spending and retention** as main components. We expect you to link this information in order to find **what triggers higher loyalty scores** and you can use any kind of modeling, including recommendation system, but not exclusively. You can be as creative as you want!"*

- By looking at 'historical_transactions.csv' and 'new_merchant_transactions.csv,' we can see that the historical transactions are those occurred **before the "reference date"** and new merchant transactions the ones that occurred **after the reference date** ("month lag to reference date").

### - Our understanding of the situation:

 - Based on the data in historical_transactions.csv, Elo picked new merchants to recommend for each card holder.
 - The date when Elo began providing recommentations is called the 'reference date'.
 - The recommended merchant data is not provided (so we can not figure out the recommendation algorithm Elo uses).
 - After the reference date, for each card ID Elo gathered transaction history for all new merchants that appeared on the card.
 - By comparing each card's new merchant activity and the secret list of the merchants recommended by Elo, the loyalty score was calculated.
 
 
### - Goal:
 Evaluate Elo's recommendation algorithm by **predicting the loyalty score** and finding out which features are correlated to the loyalty score.


### - Procedure: 
We were choosing the following procedure in order to accomplish our goal. The illustration does not show the selection of the right parameters for each model (Hyperparameter Tuning), which turned out to be crucial for the success of our models. We focus on this in chapters 3.2.1 and 3.3.1.

![](https://i.imgur.com/aNshtKq.png)

<a id="2"></a> <br>
## 2. EDA

<a id="2.1"></a> <br>
## 2.1 Data Load

Loading different libraries and packeges into the notebook. Some modules are imported in the relevant cells to make it more understandable and readable. 

In [ ]:
# Importing libraries and packages
import numpy as np ; import pandas as pd 
import matplotlib.pyplot as plt ; import seaborn as sns; import graphviz
from matplotlib.pyplot import figure;
import matplotlib.ticker as ticker
import warnings ;import time ;import sys ;import datetime;import gc; warnings.simplefilter(action='ignore')
import plotly.offline as py;py.init_notebook_mode(connected=True);import plotly.graph_objs as go; import plotly.tools as tls

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, learning_curve, validation_curve
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb 

On of the first necessary steps was it, to challenge the **memory problem**. We reduced the memory usage by this function from Ashish Gupta.  https://www.kaggle.com/roydatascience/elo-stack-interactions-on-categorical-variables

It basicially changes the data types and helped us to reduce our memory usage by up to 56%.

In [ ]:
# Copied function to reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

Loading the data into the Kernel:

In [ ]:
%%time
new_transactions = pd.read_csv('../input/new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('../input/historical_transactions.csv', parse_dates=['purchase_date'])

org_train = pd.read_csv('../input/train.csv')
org_test = pd.read_csv('../input/test.csv')

Reducing the memory usage with the predefined funtion.

In [ ]:
# Reduce memory usage
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

<a id="2.2"></a> <br>
### 2.2 Visualization

In [ ]:
print('historical_transactions contains ' + str(len(historical_transactions)) + ' transactions ' + 'for ' + str(len(historical_transactions.card_id.unique())) + ' card_ids')
print('new_marchants_transactions contains ' + str(len(new_transactions)) + ' transactions ' + 'for ' + str(len(new_transactions.card_id.unique())) + ' card_ids')

Just looking at org_train does not shape an understanding of data, so the next visualizations bring important aspects of that data into focus for further analysis.

Firstly, let's take a look on the distribution of feature_1, feature_2 and feature_3. We can make the conclussion that feature_3 has to be binarized and the best approach in dealing with feature_1 and feature_2 is an implementation of dummies.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (14, 4));
org_train['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='#099EE8', title='feature_1', fontsize = 5);
org_train['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='red', title='feature_2', fontsize = 5);
org_train['feature_3'].value_counts().sort_index().plot(kind='pie', ax=ax[2], colors = ['#099EE8', '#FFE600'], title='feature_3', explode=[0,0.1], autopct='%1.1f%%',shadow=False)

What we can also do is to directly plot a correlation matrix. The correlation between different variables in a dataframe is very low, meaning that we are lacking the data in the org_train for accurate predictions.

In [ ]:
train_corr_matrix = org_train.corr()
# Generate mask for the upper triangle
mask = np.zeros_like(train_corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(10, 5))
cmap = sns.diverging_palette(10, 255)
sns.heatmap(train_corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Now, lets have a look at both org_train and org_test. The column first_active_month shows the month of the first purchase for a card_id. Basically, it makes sense to compare the number of first purchases in a month for train and test data.

From the following graph the next information is discovered:

- Test dataset is highly correlated with the train date having the same distribution
- Test dataset has approximately twice less of observations

In [ ]:
d1 = org_train['first_active_month'].value_counts().sort_index()
d2 = org_test['first_active_month'].value_counts().sort_index()
data = [go.Scatter(x=d1.index, y=d1.values, name='train', mode = 'lines+markers', marker={'color': 'red'}), go.Scatter(x=d2.index, y=d2.values, name='test', mode = 'lines+markers', marker={'color': '#099EE8'})]
layout = go.Layout(dict(title = "Counts of first active",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Count'),
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

Loyalty numerical score is calculated two months after historical and evaluation period. 
Let's plot a histogram of loyalty score to understand what are we dealing with. From the histogram and descriptive statistics summary we observe that the **most of the output variables are flactuating around 0**, however the minimum loyalty score observed in the train dataset is -33.21 and the maximum reached 17.96. In comparison to 2nd quantile (-0.02) that is more robust to outliers, the mean is -0.39.

In [ ]:
# Descriptive statistics summary
print(org_train['target'].describe())
# Histogram
plt.figure(figsize=(12,8))
sns.distplot(org_train.target.values, bins=50, color="red", vertical = False, kde=True, kde_kws={"color": "#FFE600"})
plt.title("Histogram of Loyalty score", fontsize = 18)
plt.ylabel('Loyalty score', fontsize=12)
plt.show()

<a id="2.3"></a> <br>
### 2.3 Feature Engineering

*“Feature engineering is the process of **transforming raw data into features** that better represent the underlying problem to the predictive models, resulting in improved model accuracy on unseen data.”
— Dr. Jason Brownlee*

Genrally speaking,  data cleaning can be seen as a process of subtraction and **feature engineering as a process of addition**. Often the feature engineering is  one of the most valuable tasks in order to improve model performance, for three reasons:

- Isolate and highlight key information, which helps the algorithms "focus" on what’s important
- Use domain expertise
- Understand the "vocabulary" of feature engineering, you can bring in other people’s domain expertise!

The first step of dealing with columns containing dates is to change the date format. There is a method(pandas.to_datetime) in pandas library that converts argument to datetime.


In [ ]:
# Read files, change date format and calculate 'elapsed_time'
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df




Taking the first look into data, there are some categorical columns and hence, they have to be binarized. The function below is the first step of dealing with such columns.

In [ ]:
# Binarize categorical values
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

org_train = read_data('../input/train.csv')
org_test = read_data('../input/test.csv')

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [ ]:
# Dummy encoding for catogories
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

In [ ]:
# Purchase_month format to month
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

The aggregation of the different transcations for every 'card_id' turned out to be crucial to the success of the project. As seen in the procedure in the beginning of our project, we changed the aggregation of the different rows many times after we evaluated the model. 

As you will see in the correlation analysis of our features, tryining to find features which are really **correlating to the target**, appeared almost as an unsolvable task for us. What really helped us to find valuable features, was the given 'feature_importance' by the LightGBM trees at the end of the modelling chapter.

In [ ]:
#'nunique' -> Returns number of unique elements in the group
def aggregate_data(transactions):
    transactions.loc[:, 'purchase_date'] = pd.DatetimeIndex(transactions['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    # Aggregate columns by:
    aggregations = {
        'category_1': ['sum', 'mean'],'category_2_1.0': ['mean'],'category_2_2.0': ['mean'],'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],'category_2_5.0': ['mean'],'category_3_A': ['mean'],'category_3_B': ['mean'],'category_3_C': ['mean'],
        'merchant_id': ['nunique'],'merchant_category_id': ['nunique'],'state_id': ['nunique'],'city_id': ['nunique'],'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'], 'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'], 'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max']
        }
    
    # Group rows by 'card_id'
    agg_transactions = transactions.groupby(['card_id']).agg(aggregations)
    agg_transactions.columns = ['_'.join(col).strip() for col in agg_transactions.columns.values]
    agg_transactions.reset_index(inplace=True)
    
    #
    data = (transactions.groupby('card_id').size().reset_index(name='transactions_count'))
    agg_transactions = pd.merge(data, agg_transactions, on='card_id', how='left')
    return agg_transactions

In [ ]:
# Aggregate historical_transactions
hist_agg = aggregate_data(historical_transactions)
hist_agg.columns = ['hist_' + c if c != 'card_id' else c for c in hist_agg.columns]

In [ ]:
# Aggregate new_transactions
new_agg = aggregate_data(new_transactions)
new_agg.columns = ['hist_' + c if c != 'card_id' else c for c in new_agg.columns]

Finally we choosed a 'left-join' to merge every unique 'card_id' with its aggregated historical and new transactions. As we can see, that reduced the number of rows in our final_train set down to 0.01%.

In [ ]:
# Join with train- and test-dataset
final_train = pd.merge(org_train, hist_agg, on='card_id', how='left')
final_test = pd.merge(org_test, hist_agg, on='card_id', how='left')

final_train = pd.merge(final_train, new_agg, on='card_id', how='left')
final_test = pd.merge(final_test, new_agg, on='card_id', how='left')

print("Train original:", org_train.shape); print("Test original:", org_test.shape)
print("Historical Transactions:", historical_transactions.shape); print("New Transactions:", new_transactions.shape); 
print("Test merged:", final_train.shape); print("Train merged:", final_test.shape)
print("Number of Rows through Aggregation  {:0.2f}%".format(final_train.shape[0]/(historical_transactions.shape[0] + new_transactions.shape[0])))

That leads us to our final data sets to work with. We created one complete data set ( 'Full load' ) and a sample data set ( 'Data to work with') to reduce computation times and make working more convinient.

In [ ]:
# Get features
features = [c for c in final_train.columns if c not in ['card_id', 'first_active_month','target']]
categorical_feats = [c for c in features if 'feature_' in c]

# Full load
target= final_train.target
train = final_train[features]
test = final_test[features]

# Data to work with 
work_target=final_train.target
work_data=final_train.sample(n=10000)
work_train=final_train[features]

# Training and test-dataset
X_train,X_test,y_train,y_test=train_test_split(work_train,work_target,test_size=0.33,random_state=42)
print("X_train:",X_train.shape," y_train:",y_train.shape," X_test:",X_test.shape," y_test:",y_test.shape)

Time to have a look an the correlation of those new features:

In [ ]:
# Compute correlation matrix
train_corr = final_train.corr()

# Generate mask for the upper triangle
mask = np.zeros_like(train_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(train_corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

To make it a little more clear, we choosed the top 10 featueres with the highest correlation to the target. Unfortunately the observed correlations are super low. 

In [ ]:


# Get correlation of features to target
corr_matr=final_train.corr().abs().sort_values(by=['target'],ascending=False)['target'];
corr_matr.head(10)

<a id="2.4"></a> <br>
### 2.4 Pre-Processing

To train and validate our models in the following chapters we had to do some preprocessing. We choosed three different approaches and created various of our training data for each method:
- Filling NaN's with the median value - > Sklearn trees do not except missing values
- Scaling the data -> turned out to be useless, as the theory suggested 
- Removing the rows with outliers ( target value < -31 (around 10 times the standard deviation of the target))
     -> Gave us the most exciting insights 

<a id="2.4.1"></a> <br>
### 2.4.1 Missing Values

In [ ]:
# checkin data set for NaN's
X_train.isna().sum().sum()

In [ ]:
# Fill NaN's with median value
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='median')
imputer.fit(X_train)

# For training data
X_train_numpy=imputer.transform(X_train)
X_train_filled=pd.DataFrame(X_train_numpy)

# For test data
X_test_numpy=imputer.transform(X_test)
X_test_filled=pd.DataFrame(X_test_numpy)

# Checkin data set for NaN's
X_train_filled.isna().sum().sum()

<a id="2.4.2"></a> <br>
### 2.4.2 Scaling

In [ ]:
# Creating scaled data sets
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(); train_scale=X_train; test_scale=X_test
X_train_scaled = scaler.fit_transform(train_scale)
X_train_scaled = pd.DataFrame(X_train_scaled,columns=features)
X_test_scaled = scaler.fit_transform(test_scale)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=features)

In [ ]:
# Some models required a multiclass label instead of a continous one
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_test_encoded = lab_enc.fit_transform(y_test)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train_encoded))

<a id="2.4.3"></a> <br>
### 2.4.3 Outliers

Based on the visualisation of Loyalty score, **1% of outliers** was identified. Let's try now to train the model on data without outliers and compare the results.

In [ ]:
org_train = pd.read_csv('../input/train.csv')
org_test = pd.read_csv('../input/test.csv')

In [ ]:
# Adding a column to classify outliers
identify_outlier_train = org_train.assign(outliers = org_train.target)
identify_outlier_train.shape

identify_outlier_train.loc[identify_outlier_train.target < -31, 'outliers'] = 1
identify_outlier_train.loc[identify_outlier_train.target > -31, 'outliers'] = 0

# Only outliers
dirty_train = identify_outlier_train[identify_outlier_train.outliers == 1]
# Without outliers
clean_train = identify_outlier_train[identify_outlier_train.outliers == 0]

# Training and test data without outliers
clean_train = clean_train.drop(['first_active_month','card_id'],axis=1)
clean_target = clean_train['target']
X_train_clean,X_test_clean,y_train_clean,y_test_clean=train_test_split(clean_train,clean_target,test_size=0.33,random_state=42)

print('Number of outliers in train data is: ' + str(len(dirty_train)))
print('Number of non-outliers in train data is: ' + str(len(clean_train)))
print('Proportion of outliers in train dataset is {:0.2%}'.format((len(dirty_train)/len(identify_outlier_train))))

<a id="3"></a> <br>
## 3. Model Training

#### Evaluation of Models

In order to measure the quality of fit  and to evaluate the models in the following chapter, we defined the root-mean-square-error (**RMSE**). 
According to the ISLR the (R)MSE is small if the predicted response is close to the true responses and large if there is a substantial difference between the predicted and the true response.

To compare our results we created a dummy-model, which is predicting the average of the giving targets by the training set. With an RMSE of 3.875 the is only 6.5% worse than the leading score on the competitions leaderbord (Team Name: senkin 13, Score: 3.637, Date 28.01.2019).

In [ ]:
# define function to calculate RMSE
def rmse(y_test,prediction):
    rmse=np.sqrt(mean_squared_error(y_test,prediction))
    return rmse;

In [ ]:
# create a dummy prediciton with the average, in order to compare the model to the most simplest one
y_dummy=pd.Series(data=np.full((y_test.shape),np.average(target)))
rmse_dummy=rmse(y_test,y_dummy)
print('Dummy-Model RMSE: {:0.2f}'.format(rmse_dummy))

In [ ]:
# define function in order to calculate RMSE and Improvement of Model
def evaluation(model,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test):
    """
   This function evaluates the model, calculates the RMSE and measures its imporvement in comparison to the dummy-prediction.
   If no specific Training and Test-set is given, it defaults to X_train,y_train,X_test and y_test.
   """
    model.fit(X_train,y_train)
    model_pred=model.predict(X_test)
    rmse_new=rmse(y_test,model_pred)
    #print("Number of features used:",model.coef_!=0 )
    print("Model:",model.__class__.__name__,";",'RMSE: {:0.2f}'.format(rmse_new),";",'Improvement of: {:0.2f}%'.format(100 * (rmse_dummy- rmse_new) / rmse_new))

In [ ]:
# define RMSE for sklearn-models
from sklearn.metrics import make_scorer
rmse_scorer = make_scorer(rmse, greater_is_better=False)

<a id="3.1"></a> <br>
## 3.1 Linear Models

As a first approach to predict the loyalty score for the elo customers, we choosed Linear Models. Sklearn provides a wide range of linear models, which are easy to implement. In comparison to the usual LinearRegression, we tested the Lasso, Ridge and ElasticNet Regressor. Since we have a wide range of features, our motivation was it to use L1/L2 regularization in order select the optimal features. In the following the main charaktertics of the models are summed up:

**Lasso (L1):**
- L1 regularization adds a penalty equal to the sum of the absolute value of the coefficients
- Shrinks some parameters to zero. Hence some variables will not play any role in the model, L1 regression can be seen as a way to select features in a model. 
- The optimization objective for Lasso is: *(1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1*
- alpha : float, optional "Constant that multiplies the L1 term. Defaults to 1.0. alpha = 0 is equivalent to an ordinary least square, solved by the LinearRegression object." (Source: sklean)

**Ridge (L2):**
- L2 regularization adds a penalty equal to the sum of the squared value of the coefficients
- Forces the parameters to be relatively small, the bigger the penalization, the smaller (and the more robust) the coefficients are.
- Minimizes the objective function: *||y - Xw||^2_2 + alpha * ||w||^2_2*
- alpha : {float, array-like}, shape (n_targets): Regularization strength; must be a positive float. Regularization improves the conditioning of the problem and reduces the variance of the estimates. (Source: sklearn)

**Elastic Net (L1/L2):**
- Linear regression with combined L1 and L2 priors as regularizer.
- Minimizes the objective function: *1 / (2 * n_samples) * ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2*
- alpha = a + b and l1_ratio = a / (a + b). Specifically, l1_ratio = 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable, unless you supply your own sequence of alpha (Source: sklearn)

In [ ]:
# Testing Linear Models on Dataset
# https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet

linReg=LinearRegression()
logReg=LogisticRegression()
lasso=Lasso(alpha=0.9)
ridge=Ridge(alpha=0.9) # higher the alpha value, more restriction on the coefficients; 
                       # low alpha > more generalization, coefficients are barely
elasNet=ElasticNet(0.5)

for reg in (linReg,lasso,ridge,elasNet):
    evaluation(reg,X_train=X_train_filled,X_test=X_test_filled);

In [ ]:
# add significane test
import statsmodels.api as sm
import statsmodels.formula.api as smf

est = smf_linReg = smf.ols('target ~ feature_1 + feature_2 + feature_3', final_train).fit()
est.summary().tables[1]

As seen in the results, the linear models did not really work on the given data set. We did not expect something else, because we do not have any real correlation between the target and the features.

<a id="3.2"></a> <br>
## 3.2 Decision Trees

After the linear models were quite unsuccesfull we decided to go for **decision Trees**. Firsty we trained our data sets on the sklearn trees, later on we tried our luck with the **LighGBM**. Decision Tree's in general bring the following advantages with them:

- Decision trees implicitly perform variable screening or **feature selection**. When we fit a decision tree to a training dataset, the top few nodes on which the tree is split are essentially the most important variables within the dataset and feature selection is completed automatically.

- Decision trees require relatively little effort from users for data preparation. The Decision trees structure remains the same, with or without transformation. Also missing values will not prevent splitting the data for building trees. Decision trees are also **not sensitive to outliers** since the splitting happens based on proportion of samples within the split ranges and not on absolute values.

- **Nonlinear relationships** between parameters do not affect tree performance. As mentioned earlier, highly nonlinear relationships between variables will result in failing checks for simple regression models and thus make such models invalid. Decision trees do not require any assumptions of linearity in the data. 

- And after all: Decision trees are **easy to interpret** and explain to executives!


Source: http://www.simafore.com/blog/bid/62333/4-key-advantages-of-using-decision-trees-for-predictive-analytics

#### Decision Tree

In [ ]:
# Training Decision Tree and first Evaluation of Performance
tree1 = DecisionTreeRegressor(max_depth=3)
evaluation(tree1, X_train = X_train_filled, X_test = X_test_filled)

# Visualize simple decision tree
dot_tree = tree.export_graphviz(tree1, out_file=None)
tree1_graph = graphviz.Source(dot_tree); tree1_graph

#### Decision Tree Validation Curve

In the following we plotted a **validation curve**, which is showing the performance of a decision tree on the training- and on the test datas et.
While the training score keeps improving, the test score decreases after a while. This is due to **overfitting**. A highly complex tree (deep tree) is not as able anymore to **generalize** on unknown Test data.

In [ ]:
# Evaluation Training- and Test Error
# Create CV training and test scores for various training set sizes
depth_range = np.linspace(1, 10, 10) # Range of depth for Decison Tree's

train_scores,test_scores = validation_curve(DecisionTreeRegressor(),X_train_filled, y_train,
                                            param_name ="max_depth", param_range = depth_range, cv=3 ,
                                            scoring=rmse_scorer, n_jobs=-1)

# Create means and standard deviations of training set scores
train_scores_mean = np.mean(train_scores, axis=1); test_scores_mean = np.mean(test_scores, axis=1)

# Calculating optimal regularization parameter
i_depth_range_optim = np.argmax(test_scores_mean)
depth_range_optim = depth_range[i_depth_range_optim]
print("Optimal regularization parameter : %s" % depth_range_optim )
tree2=DecisionTreeRegressor(max_depth=depth_range_optim); evaluation(tree2,X_train=X_train_filled,X_test=X_test_filled)

# Plotting the Validation Curve
figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Validation Curve with DecisionTreeRegressor - Complexity");plt.xlabel("Complexity/max_depth");plt.ylabel("RMSE")
plt.plot(depth_range,train_scores_mean,  label="Training score",linewidth=4,color='blue') ;
plt.plot(depth_range,test_scores_mean, label="Cross-validation score",linewidth=4,color='red')
plt.vlines(depth_range_optim, plt.ylim()[0], np.max(test_scores_mean), color='k',linewidth=2,linestyles='--',label='Optimum on test')
plt.legend(loc="best"); plt.show()

** Decision Tree Learning Curve**

After the evaluation of the optimum depth for tree, we tried to find out how much data the tree actually needs to perform. As seen below, the performance of the DecisionTreeRegressor does not really improve with an increasing trainind data size. 

In [ ]:
# Evaluation Training Size
# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(DecisionTreeRegressor(max_depth=3), 
                                                        X_train_filled, y_train,cv=3,scoring=rmse_scorer, n_jobs=-1, 
                                                        # 20 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 20))

# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(train_sizes, train_mean, '--', color='blue',  label="Training score")
plt.plot(train_sizes, test_mean, color= 'red', label="Cross-validation score")
# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")
# Create plot -> change y-label
plt.title("Learning Curve with DecisionTreeRegressor - Training Size")
#y_labels = ax.get_yticks()
#ax.yaxis.set_major_formatter(ticker.)
plt.xlabel("Training Set Size"), plt.ylabel("RMSE"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

### Radom Forrest
After modelling single tree's, the next logical step is the **RandomForrestRegressor**. This one builds an **esemble** of decision trees, which are training with the **bagging** method. The idea of bagging is, that a combination of many so called **"weak learners"** increases the final result.  
Furthermore a Random Forest adds  randomness to the model, while growing the trees. Instead of searching for the most important feature while splitting a node, it searches for the best feature among a **random subset of features**. This results in a wide diversity that generally results in a better model.

Source: https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd

In [ ]:
# Create first random forrest 
from sklearn.ensemble import RandomForestRegressor

ranFor1 = RandomForestRegressor(max_depth=3, random_state=0,
                            n_estimators=20)
evaluation(ranFor1,X_train=X_train_filled,X_test=X_test_filled)

**RandomForrest Validation Curve**

Same like we evaluated the depth for single tree, we analyzed the number of trees for the RandomForrest.

In [ ]:
# Evaluation of n_estimators = n_trees
# Create CV training and test scores for various training set sizes
estimator_range = np.arange(2, 40, 2)
train_scores,test_scores = validation_curve(RandomForestRegressor(max_depth=3),X_train_filled, y_train,
                                                          param_name="n_estimators", param_range=estimator_range,
                                                          cv=3,scoring=rmse_scorer, n_jobs=-1)

# Create means and standard deviations of training set scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

i_estimator_range_optim = np.argmax(test_scores_mean)
estimator_range_optim = estimator_range[i_estimator_range_optim]
print("Optimal regularization parameter :",estimator_range_optim )
ranFor2=RandomForestRegressor(n_estimators=estimator_range_optim)
evaluation(ranFor2,X_train=X_train_filled,X_test=X_test_filled)

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.title("Validation Curve with RandomForestRegressor");plt.xlabel("Number of Trees");plt.ylabel("RMSE")
plt.plot(estimator_range,train_scores_mean,  label="Training score",linewidth=4,color='blue') ;
plt.plot(estimator_range,test_scores_mean, label="Cross-validation score",linewidth=4,color='red')
plt.vlines(depth_range_optim, plt.ylim()[0], np.max(test_scores_mean), color='k',linewidth=2,linestyles='--',label='Optimum on test')
plt.legend(loc="best"); plt.show()

<a id="3.2.1"></a> <br>
### 3.2.1 GridSearch

### HyperParameter Tuning

In the following we optimized the **hyperparamter** of the RandomForrest with **GridSearch**. The best way to think about hyperparameters is like the settings of an algorithm that can be adjusted to optimize performance. While model parameters are learned during training , such as the slope and intercept in a linear regression ,  hyperparameters must be **set before training**. In the case of a random forest the following ones are critical:

- **n_estimators** = number of trees in the foreset
- **max_features** = max number of features considered for splitting a node
- **max_depth** = max number of levels in each decision tree
- **min_samples_split** = min number of data points placed in a node before the node is split
- **min_samples_leaf** = min number of data points allowed in a leaf node
- **bootstrap** = method for sampling data points (with or without replacement)

We used GridSearch to find the optimum parameters for RandomForrest.

In [ ]:
from sklearn.model_selection import GridSearchCV
ranFor3=RandomForestRegressor()

# Create the parameter grid based on the results of random search 
param_grid_ranFor = {
    'bootstrap': [True], 'max_depth': [2,8], 'max_features': [2,10],
    'min_samples_leaf': [2,20], 'min_samples_split': [4,20], 'n_estimators': [10,20]
}
# Instantiate the grid search model
grid_search_ranFor = GridSearchCV(estimator = ranFor3, param_grid = param_grid_ranFor, 
                                  cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

# Fit the grid search to the data
grid_search_ranFor.fit(X_train_filled,y_train);

# Best Parameters of GridSearch
best_param_ranFor=grid_search_ranFor.best_params_ ; print(best_param_ranFor)

# Evaluation of RandomForrest with GirdSearch Parameters
ranFor3 = RandomForestRegressor(**best_param_ranFor) ; evaluation(ranFor3,X_train=X_train_filled,X_test=X_test_filled)

Unfortunately the even an simple GridSearch took several, even for only a small part of the data set and for a small number of parameters. 
For that reason we decided to do the GridSearch ones for demonstration (with only a small grid of parameters)  and then focus on the LightGBM Models which turned out to be way for efficient.

<a id="3.3"></a> <br>
## 3.3 LightGBM

What is Light GBM?
- According to LightGBM'S documentation: "LightGBM is a **gradient boosting framework** that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:
- Faster training speed and higher efficiency.
- Lower memory usage.
- Better accuracy.
- Support of parallel and GPU learning.
- Capable of handling large-scale data."

How it differs from other tree based algorithm?
- Light GBM **grows tree vertically/ leaf-wise** while other algorithm grows trees **horizontally/ leaf-wise** . It will choose the leaf with max delta loss to grow. When growing the same leaf, Leaf-wise algorithm can reduce more loss than a level-wise algorithm. 


![](https://i.imgur.com/kLlL7YF.png)

( Source: http://mlexplained.com/2018/01/05/lightgbm-and-xgboost-explained/)

Level-wise training can be seen as a form of regularized training since leaf-wise training can construct any tree that level-wise training can. Therefore, leaf-wise training is more **prone to overfitting but is more flexible**. This makes it a better choice for large and wide datasets.

https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

### Parameters LightGBM
LightGBM's documentation differs between learning control and core paremters:

**Learning Control Parameters**
- **max_depth**: Describes the maximum depth of tree. Used to handle model overfitting. 
- **min_data_in_leaf**: Minimum number of the records a leaf may have. The default value is 20, optimum value. It is also used to deal overfitting.
- **feature_fraction**: % of parameters selected randomly in each iteration for building trees.
- **bagging_fraction**: Specifies the fraction of data to be used for each iteration and is generally used to speed up the training and avoid overfitting.
- **early_stopping_round**: Stops training if one metric of one validation data doesn’t improve in last - early_stopping_round rounds. 
- **lambda**: Specifies regularization. Typical value ranges from 0 to 1.
- **min_gain_to_split**: Describe the minimum gain to make a split. It can used to control number of useful splits in tree.

**Core Parameters**
- **task**: Specifies the task you want to perform on data. It may be either train or predict.
- **application**: Specifies the application of your model, whether it is a regression problem or classification problem. 
- **boosting**: Defines the type of algorithm you want to run, default=gdbt
  - gbdt: traditional Gradient Boosting Decision Tree
  - rf: random forest
 - **num_boost_round**: Number of boosting iterations, typically 100+
- **learning_rate**: Determines the impact of each tree on the final outcome. GBM works by starting with an initial estimate which is updated using the output of each tree. The learning parameter controls the magnitude of this change in the estimates. Typical values: 0.1, 0.001, 0.003…
- **num_leaves**: number of leaves in full tree, default: 31
- **device**: default: cpu, can also pass gpu

In [ ]:
# Default parameters for LightGBM
param1 = {'application': "regression", "boosting": "gbdt", "metric": 'rmse', 'max_depth': 3,  
          'learning_rate': 0.1, 'num_leaves':31, 'min_data_in_leaf': 20, "random_state": 2019,
          'min_gain_to_split':0.5,      # default =0
          'feature_fraction': 0.5,      # default =1 
          'bagging_fraction': 0.5,      # default =1 
         }

In [ ]:
# define function in order to calculate RMSE and Improvement of Model
def lgb_evaluation(model,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test):
    pred_lgb_model=model.predict(X_test)
    rmse_new=rmse(y_test,pred_lgb_model)
    print("Model: LGB;",'RMSE: {:0.2f}'.format(rmse_new),";",'Improvement of: {:0.2f}%'.format(100 * (rmse_dummy- rmse_new) / rmse_new))

In [ ]:
#Setting up the first LGBM Model
rounds = 10
training_data = lgb.Dataset(data = X_train, label = y_train, params = param1, 
                          categorical_feature = categorical_feats, free_raw_data = False)
#Training the Model
%time lgb1 = lgb.train(train_set = training_data, params = param1, num_boost_round = rounds)

#Making Predictions
pred_lgb1=lgb1.predict(X_test)

#Evaluation of first LGB
lgb_evaluation(model=lgb1)

The result of the first LGBM tree shows, that without specific parameters and intense training, there is not much difference to a usual decision tree. Let's see if we can find the optimum parameters more elegant than in Sklearn and finally get a good result.

<a id="3.3.1"></a> <br>
### 3.3.1 Bayesian Optimization


According to TowardsDataScience, the challenge with hyperparameter optimization is that **evaluating the objective function is extremely expensive**. For each different set of parameters, we need to train the model, make predictions and calculate the validation set. For complex models and large numbers of parameters that can take up to days.
Methods like GridSearch or RandomSearch are relatively inefficient, because they do not choose the next hyperparameters to evaluate based on previous results. They are **uninformed by past evaluations**, and as a result, often spend a significant amount of time evaluating “bad” hyperparameters.

Bayesian optimization finds the value that **minimizes an objective function** by building a surrogate function (probability model) **based on past evaluation** results of the objective. 
The surrogate is cheaper to optimize than the objective, so the next input values to evaluate are selected by applying a criterion to the surrogate (often Expected Improvement). 

The concept is: limit expensive evaluations of the objective function by choosing the next input values based on those that have done well in the past. Basically there are four parts of optimization problem:
- **Objective Function**: what we want to minimize, in this case the validation error of a machine learning model with respect to the hyperparameters
- **Domain Space**: hyperparameter values to search over
- **Optimization algorithm**: method for constructing the surrogate model and choosing the next hyperparameter values to evaluate
- **Result history**: stored outcomes from evaluations of the objective function consisting of the hyperparameters and validation loss

(Source: https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)

Bayesian Optimization to find optimum Parameters:

In [ ]:
from bayes_opt import BayesianOptimization

# Define Bayes Optimization function for LGBM
def bayes_parameter_opt_lgb(training, testing, init_round=5, opt_round=10, n_folds=4, random_seed=2019, 
                            n_estimators=100, learning_rate=0.05, output_process=False):
    
    # Training data
    training_data = lgb.Dataset(data= train, label= target, categorical_feature = categorical_feats, free_raw_data=False)
    
    # Parameters to optimize
    def lgb_evaluation(learning_rate, num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1,min_data_in_leaf,min_split_gain):
        
        params = {'application':'regression','num_iterations': n_estimators,  
                  'early_stopping_round':100, 'metric':'rmse'}
        # Bayes opt's outputs are always float
        params['learning_rate'] = max(min(feature_fraction, 1), 0)
        params["num_leaves"] = int(round(num_leaves)) # Rounding of parameters did not work
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['min_split_gain'] = min_split_gain
        cv_result = lgb.cv(params, training_data, nfold=n_folds, seed=random_seed, stratified=False, 
                           verbose_eval =200, metrics=['rmse'])
        return max(cv_result['rmse-mean'])
   
    # Parameter Range
    lgbBO = BayesianOptimization(lgb_evaluation, {'learning_rate': (0.0001, 0.1),
                                            'num_leaves': (150, 300),
                                            'feature_fraction': (0.01, 0.2),
                                            'bagging_fraction': (0.2, 1),
                                            'max_depth': (1, 3),
                                            'lambda_l1': (0, 2),
                                            'min_data_in_leaf': (200,400),
                                            'min_split_gain': (0.001, 0.2)})
    # Optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # Output optimization process to CSV
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result1.csv")
    
    # return best parameters
    return lgbBO.max

In [ ]:
# Using optimization function on training data
%time 
opt_params_lgb = bayes_parameter_opt_lgb(train, target)
param1=opt_params_lgb['params'];param1

In [ ]:
# Results of different rounds of Bayesian Optimization
param1 = {"boosting": "gbdt", 'objective':'regression',"metric": 'rmse', "random_state": 2019, "verbosity": -1,
          'bagging_fractions':0.7768, 'feature_fraction':0.3249, 'lambda_l1':0.0460, 'learning_rate':0.067,
          'max_depth':3, 'min_data_in_leaf':176, 'min_split_gain':0.0297, 'num_leaves':77}
param2 = {'num_leaves': 111, 'min_data_in_leaf': 149,'objective':'regression','max_depth': 9,
         'learning_rate': 0.005, "boosting": "gbdt", "feature_fraction": 0.7522, "bagging_freq": 1,
         "bagging_fraction": 0.7083 , "bagging_seed": 11, "metric": 'rmse', "lambda_l1": 0.2634,
         "random_state": 133, "verbosity": -1}
param3 = {"boosting": "gbdt", 'objective':'regression',"metric": 'rmse', "random_state": 2019, "verbosity": -1,
          'bagging_fractions':1, 'feature_fraction':0.1, 'lambda_l1':1.0, 'learning_rate':0.001,
          'max_depth':2, 'min_data_in_leaf':250, 'min_split_gain':0.001, 'num_leaves':160}
param4 = {"boosting": "gbdt", 'objective':'regression',"metric": 'rmse', "random_state": 2019, "verbosity": -1,
          'bagging_fractions':0.7, 'feature_fraction':0.02, 'lambda_l1':1.75, 'learning_rate':0.05,
          'max_depth':3, 'min_data_in_leaf':305, 'min_split_gain':0.01, 'num_leaves':300}

After finding (optimal) hyperparameters in an incedible **short amount of time** (in comparison to GridSearch) we defined a function to run the LGBM.

The function uses KFold to create indizes for **cross-validation**. Each fold is used once for validation and k - 1 times for training. After that the LGBM is trained on each fold for a given number of rounds, whereas every round can be seen as one tree. The training stops after the validation did not improve for a given number of 'early_stopping_rounds.

In [ ]:
#train/test indices to split data in train/test sets -> each fold is then used once as a validation while the k - 1 remaining folds form the training set
folds             = KFold(n_splits=4,shuffle=True,random_state=15)
X                 = train
y                 = target
param             = param3
num_boost_rounds  = 10000
early_stopping    = 100
out_of_folds      = np.zeros(len(X))
predictions       = np.zeros(len(y))
feature_importance= pd.DataFrame()
evals_result      = {}

# for-loop to split data in n training and validation sets and perfom LGBM on every fold
for fold, (train_ind,val_ind) in enumerate(folds.split(X,y)):
    print("fold n°{}".format(fold))
    
    #training&validation sets
    training=lgb.Dataset(X.iloc[train_ind],label=y.iloc[train_ind])
    validation=lgb.Dataset(X.iloc[val_ind],label=y.iloc[val_ind])
    
    #training for each fold:
    lgb2=lgb.train(param1,training,valid_sets = [training,validation], verbose_eval=10,num_boost_round=num_boost_rounds,
                   early_stopping_rounds = early_stopping  , evals_result = evals_result)
    
    #predictions on validation fold -> OOF:"Out-of-fold" -> using k-fold validation in which the predictions from each set of folds are grouped together into one group of 1000 predictions
    out_of_folds[val_ind] = lgb2.predict(X.iloc[val_ind], num_iteration=lgb2.best_iteration)
    
    #storing in feature importance
    fold_importance= pd.DataFrame()
    fold_importance["feature"] = features
    fold_importance["importance"] = lgb2.feature_importance()
    fold_importance["fold"] = fold + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
lgb_evaluation(lgb2)

In [ ]:
lgb2.best_score

We can nicely see how the training score keeps improving while overfitting the train and the validation score stagnates.

In [ ]:
#figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
#plt.title("Validation Curve with RandomForestRegressor");plt.xlabel("Number of Trees");plt.ylabel("RMSE")
#plt.legend(loc="best"); plt.show()
ax = lgb.plot_metric(evals_result, metric='rmse')
plt.show()

<a id="3.3.2"></a> <br>
### 3.3.2 Feature Importance

LGBM provides us the importance of the features, like every other tree, without extra effort. We can clearly see that the LGBM **gained to most** when he used features which have been created by the **purchase amount** of an customer.

In [ ]:
feature_importance.sort_values(by=['importance'],ascending=False).head(10)

In [ ]:
# Group by feature
agg_features = (feature_importance[["feature","importance"]].groupby("feature").mean().sort_values(by="importance",ascending=False)[:1000].index)
best_features = feature_importance.loc[feature_importance.feature.isin(agg_features)]

# Plot feature importance
plt.figure(figsize=(12,20))
sns.barplot(x="importance",y="feature",data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (avg over folds)');plt.tight_layout()
#plt.savefig('lgbm_importances.png')

In [ ]:
predictions = lgb2.predict(test)
elo_sub_XX = pd.DataFrame({"card_id":org_test["card_id"].values})
elo_sub_XX["target"] = predictions
elo_sub_XX.to_csv("elo_subX.csv", index=False)

<a id="3.4"></a> <br>
### 3.4 Models without Outliers

Finally we train our models on the data without outliers. The results have been quite imressive, apperently the only reason why our models did not perform have been the outliers.

In [ ]:
#train/test indices to split data in train/test sets -> each fold is then used once as a validation while the k - 1 remaining folds form the training set
folds             = KFold(n_splits=5,shuffle=True,random_state=15)
X                 = clean_train
y                 = clean_target
param             = param3
num_boost_rounds  = 100
early_stopping    = 10
out_of_folds      = np.zeros(len(X))
predictions       = np.zeros(len(y))
feature_importance= pd.DataFrame()
evals_result      = {}

# for-loop to split data in n training and validation sets and perfom LGBM on every fold
for fold, (train_ind,val_ind) in enumerate(folds.split(X,y)):
    print("fold n°{}".format(fold))
    
    #training&validation sets
    training=lgb.Dataset(X.iloc[train_ind],label=y.iloc[train_ind])
    validation=lgb.Dataset(X.iloc[val_ind],label=y.iloc[val_ind])
    
    #training for each fold:
    lgb_clean=lgb.train(param1,training,valid_sets = [training,validation], verbose_eval=10,num_boost_round=num_boost_rounds,
                   early_stopping_rounds = early_stopping  , evals_result = evals_result)
    
    #predictions on validation fold -> OOF:"Out-of-fold" -> using k-fold validation in which the predictions from each set of folds are grouped together into one group of 1000 predictions
    out_of_folds[val_ind] = lgb_clean.predict(X.iloc[val_ind], num_iteration=lgb_clean.best_iteration)

    
lgb_evaluation(lgb_clean ,X_train = X_train_clean, y_train = y_train_clean,
           X_test = X_test_clean, y_test = y_test_clean );

In [ ]:
lgb_clean.best_score

Even the Linear Models are providing really good results now. 

In [ ]:
linReg=LinearRegression()
logReg=LogisticRegression()
lasso=Lasso(alpha=0.9)
ridge=Ridge(alpha=0.9) # higher the alpha value, more restriction on the coefficients; 
                       # low alpha > more generalization, coefficients are barely
elasNet=ElasticNet(0.5)

for reg in (linReg,lasso,ridge,elasNet):
    evaluation(reg ,X_train = X_train_clean, y_train = y_train_clean,
               X_test = X_test_clean, y_test = y_test_clean );

In [ ]:
#predictions = lgb_clean.predict(test)
#elo_sub_clean = pd.DataFrame({"card_id":org_test["card_id"].values})
#elo_sub_clean["target"] = predictions
#elo_sub_clean.to_csv("elo_sub_clean.csv", index=False)

<a id="4"></a> <br>
# 4. Conclusion


### Findings:

- The real challenge of the competetion is to **predict the outliers**. Whoever will manage it to include an outlier prediction into his model, will probably win the competition
- Althoug it is only around 1 %, the  **extreme outliers** (around 10 standard deviations -> -31 value) **blow out the RMSE's** of all competitors so far
- Looking at the Leaderboard ( best RMSE around 3.6) no one has accomplished that so far
- Training an data set **without outliers** gives us **RMSE's down to 0.37**, but of course only test data without outliers as well
- Looking at **the feature importance**, the amount one customer spent turned out to be the most important indicator for loyalty
- The features which stand out the most are the ones that represent **how much a customer paid and when they committed those transactions**

### Lessons learned:
 
 -  Handle the **memory usage** of your data. Chaning the format of the data as well as using samples of the data turned out to be really helpful to move the data through the pipeline
 - **Aggregate** rows from the historical data set by using min, mean, max etc. 
 - Save lots of time and use **Bayesian Optimization** to find the right parameters, instead of GridSearch
 -  Drop the columns that do not have a similar distribution of values between the train and test files. This will cause your predictions on test to be well prepared by the train data set.
 - Evaluate the **data input** for your model. It turned out over and over again, that the **data quality** is way more important for a model to learn than the algorithm itself.
 - Let the model know, which features are **categorical**
 - While working on one data project for a long time, it can be really helpful to **define functions and pipelines** to automate certain repeating steps
 
